In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score

import random
from scipy.optimize import minimize

In [ ]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2023/0_data/source_MAT_files'

dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_DEC23.csv')

dyndb = pd.read_csv(dbpath)

stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('untrunc.csv')]


In [ ]:
dyndb_csvs = (dyndb.dyn_file_name_original + '_processed_untrunc').values

In [ ]:
opt_cols = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'W_pred', 'sig_pl_pred']

for col in opt_cols:
    dyndb[col] = pd.Series(np.zeros(len(dyndb)))

In [ ]:
import pygad

In [ ]:
instance = 'rhomb_dodec_0p24_sr1000'#'XRho_0p30_sr1000'#'Design1_VF33p1'#'' #

In [ ]:
csvpath = os.path.join(stress_ser_path, f'{instance}_processed_untrunc.csv')

df = pd.read_csv(csvpath)

In [ ]:
sig_pl = dyndb[dyndb['dyn_file_name_original'] == instance]['plateau_stress_g'].values[0]#/10e6
W = dyndb[dyndb['dyn_file_name_original'] == instance]['energy_absorbed_g'].values[0]#/10e6
print(f'{sig_pl:.3e}\t{W:.3e}')

pl_end = dyndb[dyndb['dyn_file_name_original'] == instance]['end_strain_g'].values[0]
# pl_end_idx = dyndb[dyndb['Strain'] == pl_end].index

strn = np.asarray(df['Strain'])
pl_end_idx = np.where(np.isclose(pl_end, strn))[0][0]

strain = df['Strain'][:pl_end_idx]
stress_series = df['stress_bottom_gsreg'][:pl_end_idx]#/10e6


In [ ]:
A = A_guess = np.exp(random.randrange(7,12))
B = B_guess = np.exp(random.randrange(16,24))
C = C_guess = random.randrange(1,8)
m = m_guess = random.uniform(0.1, 0.65)
n = n_guess = random.randrange(0,20)


initial_guess = [A_guess, B_guess, C_guess, m_guess, n_guess]

# desired_output = stress_series
function_inputs = stress_series

In [ ]:
bounds = ([0, None],[0, None], [0,10], [0.1,0.75], [0,20])

In [ ]:
# def fitness_func(solution, strain, stress_series, sig_pl, W, solution_idx):

#     A, B, C, m, n = solution

#     stress_series_fit = A * strain**m + B*(strain / (C - strain))**n
#     # stress_series_err = np.sum((stress_series - stress_series_fit)**2)
#     stress_series_err = np.sum(np.abs(stress_series - stress_series_fit))

    
#     sig_pl_pred = np.mean(stress_series_fit[(strain >= 0.2) & (strain <= 0.4)])
#     sig_pl_err = np.abs(sig_pl_pred - sig_pl)

#     # pl_end_pred = np.where(stress_series >= 1.3*sig_pl)[0][0] #?????
#     # pl_end_pred = np.where(np.isclose(stress_series, 1.3*sig_pl, atol=1e8))[0][-1] #?????
    
#     pl_end_pred = stress_series_fit[-1:].values[0]   #?????
#     pl_end_err = np.abs(stress_series[-1:].values[0]  - pl_end_pred)

#     # W_pred = np.trapz(A * strain**m + B * (strain / (1 - strain))**n, strain)
#     W_pred = np.trapz(stress_series_fit, strain)

#     W_err = np.abs(W_pred - W)

#     total_error =  sig_pl_err + W_err + pl_end_err + stress_series_err
    
#     fitness = 1.0 / total_error

#     return fitness

In [ ]:
def fitness_func(ga_instance, solution,  solution_idx):

    global A, B, C, m, n, strain, stress_series, sig_pl, W

    stress_series_fit = A * strain**m + B*(strain / (C - strain))**n
    # stress_series_err = np.sum((stress_series - stress_series_fit)**2)
    stress_series_err = np.sum(np.abs(stress_series - stress_series_fit))

    
    sig_pl_pred = np.mean(stress_series_fit[(strain >= 0.2) & (strain <= 0.4)])
    sig_pl_err = np.abs(sig_pl_pred - sig_pl)

    # pl_end_pred = np.where(stress_series >= 1.3*sig_pl)[0][0] #?????
    # pl_end_pred = np.where(np.isclose(stress_series, 1.3*sig_pl, atol=1e8))[0][-1] #?????
    
    pl_end_pred = stress_series_fit[-1:].values[0]   #?????
    pl_end_err = np.abs(stress_series[-1:].values[0]  - pl_end_pred)

    # W_pred = np.trapz(A * strain**m + B * (strain / (1 - strain))**n, strain)
    W_pred = np.trapz(stress_series_fit, strain)

    W_err = np.abs(W_pred - W)

    # total_error =  sig_pl_err + W_err + pl_end_err + stress_series_err
    
    # fitness = 1.0 / total_error

    fitness = 1.0 / np.asarray([sig_pl_err, W_err, pl_end_err, stress_series_err])

    return fitness

In [ ]:
num_generations = 50
num_parents_mating = 4

sol_per_pop = 9
num_genes = len(initial_guess)

init_range_low = 0
init_range_high = 10e9

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

In [ ]:
initial_guess = np.array(
    [[1e7, 1e10],
     [1e7, 1e10],
     [0,5],
     [0.05, 0.75],
     [5,15]]
)

# initial_guess = np.array(
#     [[1e7, 1e7, 0, 0.05, 5],
#      [1e10, 1e10, 5, 0.75, 15]]
# )

In [ ]:
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       initial_population=initial_guess,
                      #  init_range_low=init_range_low,
                      #  init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes)

In [ ]:
ga_instance.run()

In [ ]:
function_inputs.shape

In [ ]:
# solution

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

# prediction = function_inputs[200:400].mean()
# print(f"Predicted output based on the best solution : {prediction:.3e}\tactual sig_pl:\t {sig_pl:.3e}")